In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/flood detection

/content/drive/MyDrive/flood detection


In [3]:
!pip install pyngrok
!pip install flask-ngrok
!pip install geopy
!pip install folium
!pip install webbrowser

ERROR: Could not find a version that satisfies the requirement webbrowser (from versions: none)
ERROR: No matching distribution found for webbrowser


In [ ]:
!ngrok authtoken 2bnGLQ7sEmfFw6TUd9wH5ouUBYk_25tKNCDKB1SexuRpEhMr

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from flask import Flask, flash, request, redirect, url_for, render_template
import os
from werkzeug.utils import secure_filename
import cv2
import numpy as np
from geopy.geocoders import Nominatim
import folium
from tensorflow.keras.models import load_model
from flask_ngrok import run_with_ngrok
from pyngrok import ngrok

# Remove the unnecessary import
# from keras.initializers import Orthogonal  # Removed

app = Flask(__name__)
run_with_ngrok(app)

UPLOAD_FOLDER = 'static/uploads/'
SIZE = 64

app.secret_key = "secret key"
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024

ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg', 'gif'}

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/', methods=['POST'])
def upload_image():
    error = None
    if 'file' not in request.files:
        error = "No file selected for uploading"
        return redirect(request.url)

    file = request.files['file']
    if file.filename == '':
        error = "No selected file"
        return redirect(request.url)

    if file and allowed_file(file.filename):
        filename = secure_filename(file.filename)
        file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))

        # Load model
        model = load_model('flooding1.h5')
        categories = ["Flooding", "No Flooding"]

        # Image processing logic (moved outside for clarity)
        image_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        predicted_class, pValue = process_image(image_path, model, categories)


        # Update error message based on prediction
        if predicted_class == 0:
            error = "Flood Detected"

            # Get location name from filename
            location_name = os.path.splitext(filename)[0]

            # Use Nominatim to get latitude and longitude
            geolocator = Nominatim(user_agent="map_visualizer")
            location = geolocator.geocode(location_name)
            latitude = location.latitude
            longitude = location.longitude

            # Create a map centered on the retrieved coordinates
            m = folium.Map(location=[latitude, longitude], zoom_start=10)

            # Add a marker for the location
            folium.Marker([latitude, longitude], popup=location_name).add_to(m)

            # Save the map to an HTML file
            m.save('templates/map.html')

        else:
            error = "No flood detected"

        flash('Image successfully uploaded and displayed below')
        return render_template('index.html', filename=filename, error=error)

    else:
        error = "Allowed image types are - png, jpg, jpeg, gif"
        return redirect(request.url)

def process_image(image_path, model, categories):
            nimage = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            image = cv2.resize(nimage, (SIZE, SIZE))  # Replace SIZE with your desired size
            image = image / 255.0
            prediction = model.predict(np.array(image).reshape(-1, SIZE, SIZE, 1))
            pclass = np.argmax(prediction)
            pValue = "The Signature is: {}".format(categories[int(pclass)])
            return pclass, pValue

@app.route("/map")
def map():

    return render_template("map.html", cache_time=0)


@app.route('/display/<filename>')
def display_image(filename):
    return redirect(url_for('static', filename='uploads/' + filename), code=301)

if __name__ == "__main__":
    ngrok_tunnel = ngrok.connect(5000)
    print(' * Tunnel URL:', ngrok_tunnel.public_url)
    app.run()

 * Tunnel URL: https://56ab-34-106-61-127.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://56ab-34-106-61-127.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [22/Apr/2024 13:43:09] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Apr/2024 13:43:11] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [22/Apr/2024 13:43:17] "GET /templates/about.html HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [22/Apr/2024 13:43:24] "GET /templates/index.html HTTP/1.1" 404 -
